sempre que adicionar ou mudar uma dependencia, usar pip freeze > requirements.txt


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import requests
import re

In [ ]:
dadosAcumulados = [] # lista para armazenar os dados acumulados

page = 1 # variavel para controlar a paginação
i = 1

while True: 

    url = f'https://books.toscrape.com/catalogue/page-{page}.html'

    # resposta com o conteudo da pagina
    response = requests.get(url) 
    
    # verifica se a resposta é diferente de 200, se for, sai do loop
    if response.status_code != 200: 
        break

    result = response.text 

    soup = BeautifulSoup(result, 'lxml')
    
    # busca diretamenta todas as tags li que estão dentro de uma tag ol com a classe row
    for item in soup.select('ol.row li'): 
        

        # busca tags a que tenham o atributo title
        title_aux = item.find("a", attrs={"title":True}) 
        # extrair o texto que está dentro do atributo title de cada tag a encontrada 
        title = title_aux.get('title') 

        # busca tags p que tenham a classe star-rating
        star_aux = item.find("p", class_="star-rating")
        # extrair o texto que está dentro do atributo class na posicao 1 de cada tag p encontrada(é a informação que tem a avaliação do livro) 
        star = star_aux.get('class')[1]

        # busca tags p que tenham a classe price_color
        price_aux = item.find("p", class_="price_color")
        # extrair o texto que está dentro de cada tag p encontrada 
        price = price_aux.getText()

        # busca tags p que tenham a classe instock availability
        availability_aux = item.find("p", class_="instock availability")
        # extrair o texto que está dentro de cada tag p encontrada
        availability = availability_aux.getText()

        # Passa todas as informações encontradas como um dicionario para a varivavel dadosAcumulados, contruindo uma lista de dicionarios
        dadosAcumulados.append({'title':title, 'rating':star, 'price':price, 'availability':availability})

        i = i + 1
        print("loop for: ",i)

    page = page + 1 

    print("pagina: ",page)


In [ ]:
for item in dadosAcumulados:
    print(item)


In [7]:
# Para cada item dentro da lista dadosAcumulados, crie um novo dicionário com apenas duas informações: title e stars.
starTable = [{"title": item["title"], "stars": item["rating"]} for item in dadosAcumulados]
priceTable = [{"title": item["title"], "price": item["price"]} for item in dadosAcumulados]
availabilityTable = [{"title": item["title"], "disponibility": item["availability"]} for item in dadosAcumulados]

#print(starTable)


In [ ]:
starDF = pd.DataFrame(starTable)
priceDF = pd.DataFrame(priceTable)
availabilityDF = pd.DataFrame(availabilityTable)



                                   title  stars
0                   A Light in the Attic  Three
1                     Tipping the Velvet    One
2                             Soumission    One
3                          Sharp Objects   Four
4  Sapiens: A Brief History of Humankind   Five


In [ ]:
# Inicializar a SparkSession
spark = SparkSession.builder \
    .appName("Exemplo Simples de Tabela") \
    .getOrCreate()

# Definir o schema da tabela
schema = StructType([
    StructField("id", IntegerType(), nullable=False),
    StructField("nome", StringType(), nullable=True),
    StructField("idade", IntegerType(), nullable=True),
    StructField("cidade", StringType(), nullable=True)
])

# Dados de exemplo
data = [
    (1, "João Silva", 28, "São Paulo"),
    (2, "Maria Souza", 34, "Rio de Janeiro"),
    (3, "Carlos Oliveira", 22, "Belo Horizonte"),
    (4, "Ana Santos", 45, "Curitiba")
]

# Criar DataFrame
df = spark.createDataFrame(data, schema=schema)

# Mostrar a tabela
print("Mostrando a tabela completa:")
df.show()

# Mostrar a tabela com formatação
print("\nMostrando a tabela com formatação:")
df.show(truncate=False)

# Mostrar o schema da tabela
print("\nSchema da tabela:")
df.printSchema()

spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Criar SparkSession
spark = SparkSession.builder.appName("EmptyDF").getOrCreate()

# 1. Criar DataFrame vazio com schema definido
schema = StructType([])  # Schema vazio
df = spark.createDataFrame([], schema)

# 2. Adicionar colunas uma a uma
from pyspark.sql.functions import lit

# Adicionar coluna de strings
df = df.withColumn("nome", lit(None).cast(StringType()))

# Adicionar coluna de inteiros
df = df.withColumn("idade", lit(None).cast(IntegerType()))

# Adicionar coluna de doubles com valor padrão 0.0
df = df.withColumn("salario", lit(0.0).cast(DoubleType()))

df.printSchema()
df.show()